In [1]:
import os
os.chdir("../")
%pwd

'd:\\PracticeProjects\\NLP_Insurance_intent_Detection\\nlp_intent'

In [2]:
# entity

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class ModelTrainerConfig:
    root_dir: Path
    data_path : Path
    model_ckpt : Path
    num_train_epochs : int
    learning_rate : float
    per_device_train_batch_size : int
    per_device_eval_batch_size : int
    weight_decay : float
    evaluation_strategy : str
    disable_tqdm : bool
    logging_steps : int
    log_level : str      

In [3]:
# Configuration Manager

from intent.constants import *
from intent.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        #print(CONFIG_FILE_PATH)
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath) 
        create_directories([self.config.artifacts_root])

    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments
        create_directories([config.root_dir])
        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs =  params.num_train_epochs,
            learning_rate = params.learning_rate,
            per_device_train_batch_size = params.per_device_train_batch_size,
            per_device_eval_batch_size = params.per_device_eval_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            disable_tqdm = params.disable_tqdm,
            log_level = params.log_level,
        )
        return model_trainer_config

In [4]:
# Component

import os
from intent.logging import logger
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
from datasets import load_from_disk
from sklearn.metrics import f1_score, accuracy_score
import torch

class ModelTrainer:
    def __init__(self, config:ModelTrainerConfig):
        self.config = config
    
    def compute_metrics(self, pred):
        labels = pred.label_ids
        preds = pred.predictions.argmax(-1)
        f1 = f1_score(labels, preds, average= "weighted")
        acc = accuracy_score(labels, preds)
        return {"f1" : f1, "accuracy" : acc}
    
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        num_labels = 77
        model = AutoModelForSequenceClassification.from_pretrained(self.config.model_ckpt, num_labels = num_labels).to(device)
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        train_dataset_encoded = load_from_disk(os.path.join(self.config.data_path, "train_dataset_encoded"))
        test_dataset_encoded = load_from_disk(os.path.join(self.config.data_path, "test_dataset_encoded"))

        training_args = TrainingArguments(
            output_dir = os.path.join(self.config.root_dir, "training_output_directory"),
            num_train_epochs = self.config.num_train_epochs,
            per_device_train_batch_size = self.config.per_device_train_batch_size,
            per_device_eval_batch_size = self.config.per_device_eval_batch_size,
            weight_decay = self.config.weight_decay,
            logging_steps = self.config.logging_steps,
            evaluation_strategy = self.config.evaluation_strategy,
            disable_tqdm = self.config.disable_tqdm,
            log_level = self.config.log_level,
            learning_rate = self.config.learning_rate,
           
        )
        trainer = Trainer(model = model,
                          args = training_args,
                          compute_metrics = self.compute_metrics,
                          train_dataset = train_dataset_encoded,
                          eval_dataset = test_dataset_encoded,
                          tokenizer = tokenizer
                          )
        
        trainer.train()
        model.save_pretrained(os.path.join(self.config.root_dir, f"{self.config.model_ckpt}-finetuned-banking77"))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer-finetuned-banking77"))


d:\Softwares\Anaconda3\envs\nlp_intent_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-05-17 02:22:27,898: INFO: config: PyTorch version 2.3.0+cu118 available.]


In [5]:
# Pipeline component
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config = model_trainer_config) 
    model_trainer.train()
except Exception as e:
    raise e 

[2024-05-17 02:22:28,171: INFO: common: YAML file: D:\PracticeProjects\NLP_Insurance_intent_Detection\nlp_intent\config\config.yaml loaded successfully]
[2024-05-17 02:22:28,174: INFO: common: YAML file: D:\PracticeProjects\NLP_Insurance_intent_Detection\nlp_intent\params.yaml loaded successfully]
[2024-05-17 02:22:28,175: INFO: common: created directory at: artifacts]
[2024-05-17 02:22:28,177: INFO: common: created directory at: artifacts/model_trainer]


d:\Softwares\Anaconda3\envs\nlp_intent_venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
                                                  
 10%|█         | 157/1570 [01:22<09:31,  2.47it/s]

{'eval_loss': 1.8220016956329346, 'eval_f1': 0.634310484760339, 'eval_accuracy': 0.6711038961038961, 'eval_runtime': 6.2266, 'eval_samples_per_second': 494.649, 'eval_steps_per_second': 7.869, 'epoch': 1.0}


 13%|█▎        | 200/1570 [01:44<11:28,  1.99it/s]

{'loss': 2.8009, 'grad_norm': 3.7668073177337646, 'learning_rate': 4.3630573248407646e-05, 'epoch': 1.27}


...

{'loss': 0.0252, 'grad_norm': 0.37820175290107727, 'learning_rate': 5.414012738853504e-06, 'epoch': 8.92}


                                                   
 90%|█████████ | 1413/1570 [12:28<01:03,  2.46it/s]

{'eval_loss': 0.29550763964653015, 'eval_f1': 0.9292959135889873, 'eval_accuracy': 0.9292207792207792, 'eval_runtime': 6.323, 'eval_samples_per_second': 487.114, 'eval_steps_per_second': 7.75, 'epoch': 9.0}


                                                   
100%|██████████| 1570/1570 [13:52<00:00,  1.89it/s]


{'eval_loss': 0.293503999710083, 'eval_f1': 0.9295879631152127, 'eval_accuracy': 0.9295454545454546, 'eval_runtime': 6.1165, 'eval_samples_per_second': 503.559, 'eval_steps_per_second': 8.011, 'epoch': 10.0}
{'train_runtime': 832.4287, 'train_samples_per_second': 120.166, 'train_steps_per_second': 1.886, 'train_loss': 0.5270263790324995, 'epoch': 10.0}
